In [1]:
pip install alpha_vintage

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement alpha_vintage (from versions: none)
ERROR: No matching distribution found for alpha_vintage


In [2]:
pip install alpha_vantage


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


# 1. Define a Class ScriptData which can fetch US Stock data using Alpha Vantage.

In [1]:
from alpha_vantage.timeseries import TimeSeries

In [2]:
import pandas as pd

In [3]:
ALPHAVANTAGE_API_KEY = 'OC43FD8AED2GOMCN'

In [4]:
class ScriptData:
    def __init__(self):
        self.data = {}
    
    def __getitem__(self,key):
        return self.data[key]
    
    def __setitem__(self,key,value):
        self.data[key] = value
    
    def __contains__(self,key):
        return key in self.data
        
    
    def fetch_intraday_data(self,script):
        ts = TimeSeries(key=ALPHAVANTAGE_API_KEY,indexing_type='date')
        data,meta_data = ts.get_intraday(script)
        self[script] = data
        
    
    def convert_intraday_data(self,script):
        temp = self[script]
        df = pd.DataFrame(temp)
        df = df.transpose()
        df = df.reset_index()
        df = df.rename(columns={'index':'timestamp','1. open':'open',
                                '2. high':'high','3. low':'low',
                                '4. close':'close','5. volume':'volume'
                               })
        df['open'] = df['open'].astype(float)
        df['high'] = df['high'].astype(float)
        df['low'] = df['low'].astype(float)
        df['close'] = df['close'].astype(float)
        df['volume'] = df['volume'].astype(int)
        self[script] = df
        


In [5]:
script_data = ScriptData()

In [6]:
script_data.fetch_intraday_data('GOOGL')

In [7]:
script_data.convert_intraday_data('GOOGL')

In [8]:
script_data['GOOGL']

,timestamp,open,high,low,close,volume
0,2023-06-13 19:45:00,123.8600,123.8600,123.7900,123.8100,12957
1,2023-06-13 19:30:00,123.9700,123.9700,123.8600,123.8600,3393
2,2023-06-13 19:15:00,123.9200,123.9300,123.8800,123.8800,2913
3,2023-06-13 19:00:00,123.9100,123.9100,123.9100,123.9100,785
4,2023-06-13 18:45:00,123.9932,123.9932,123.9932,123.9932,155
...,...,...,...,...,...,...
95,2023-06-12 11:30:00,122.2600,122.5000,122.2500,122.3450,866903
96,2023-06-12 11:15:00,122.1000,122.4400,122.0910,122.2600,570231
97,2023-06-12 11:00:00,122.1600,122.2771,121.9850,122.1000,562856
98,2023-06-12 10:45:00,122.1590,122.4199,122.0550,122.1600,694492


In [9]:
'GOOGL' in script_data

True

In [10]:
script_data.fetch_intraday_data('AAPL')
script_data.convert_intraday_data('AAPL')
script_data['AAPL']

,timestamp,open,high,low,close,volume
0,2023-06-13 19:45:00,183.1500,183.15,183.050,183.0799,14292
1,2023-06-13 19:30:00,183.2100,183.22,183.150,183.1600,7162
2,2023-06-13 19:15:00,183.2100,183.24,183.150,183.2400,18283
3,2023-06-13 19:00:00,183.2600,183.26,183.220,183.2200,1983
4,2023-06-13 18:45:00,183.2500,183.28,183.230,183.2600,4721
...,...,...,...,...,...,...
95,2023-06-12 12:00:00,182.3400,182.38,182.160,182.2850,929223
96,2023-06-12 11:45:00,182.4700,182.53,182.225,182.3266,1195820
97,2023-06-12 11:30:00,182.0200,182.49,182.020,182.4787,1647990
98,2023-06-12 11:15:00,181.9350,182.11,181.850,182.0150,1295263


In [11]:
'GOOGL' in script_data

True

In [12]:
'AAPL' in script_data

True

In [13]:
'NVDA' in script_data

False

# 2. Define a function called indicator1

In [22]:
def indicator1(df,timeperiod):
    df['indicator'] = df['close'].rolling(timeperiod).mean()
    new_df = df[['timestamp','indicator']].copy()
    return new_df

In [23]:
indicator1(script_data['GOOGL'],timeperiod=5)

,timestamp,indicator
0,2023-06-13 19:45:00,NaN
1,2023-06-13 19:30:00,NaN
2,2023-06-13 19:15:00,NaN
3,2023-06-13 19:00:00,NaN
4,2023-06-13 18:45:00,123.89064
...,...,...
95,2023-06-12 11:30:00,122.42534
96,2023-06-12 11:15:00,122.37834
97,2023-06-12 11:00:00,122.30100
98,2023-06-12 10:45:00,122.25100


# 3. Define a Class Strategy

In [24]:
class Strategy:
    def __init__(self,script_name):
        self.script = script_name
        self.data = {}
        self.indicator_data = None
    
    def get_script_data(self):
        script_data = ScriptData()
        script_data.fetch_intraday_data(self.script)
        script_data.convert_intraday_data(self.script)
        self.data = script_data[self.script]
        self.compute_indicator_data()
        
    def compute_indicator_data(self):
        self.indicator_data = indicator1(self.data,2)
    
    def get_signals(self):
        signals = pd.DataFrame({'timestamp':self.data['timestamp']})
        signals['signal'] = 'NO_SIGNAL'
        self.data['close'] = self.data['close'].astype(float)
        signals.loc[self.indicator_data['indicator'] > self.data['close'], 'signal'] = 'BUY'
        signals.loc[self.indicator_data['indicator'] < self.data['close'], 'signal'] = 'SELL'
        display(signals)
        

In [17]:
strategy = Strategy('NVDA')

In [18]:
strategy.get_script_data()

In [19]:
strategy.get_signals()

,timestamp,signal
0,2023-06-13 19:45:00,NO_SIGNAL
1,2023-06-13 19:30:00,BUY
2,2023-06-13 19:15:00,BUY
3,2023-06-13 19:00:00,SELL
4,2023-06-13 18:45:00,BUY
...,...,...
95,2023-06-12 12:00:00,BUY
96,2023-06-12 11:45:00,SELL
97,2023-06-12 11:30:00,SELL
98,2023-06-12 11:15:00,BUY
